In [150]:
import csv
import json
import re
import pandas as pd
import numpy as np
from pydantic import BaseModel, ValidationError
from typing import Optional, Any

In [151]:
routes = pd.read_csv('Skiturer Sunnmøre - Turer.csv')
class Route(BaseModel):
    route_id: int
    mountain_name: str
    area: str
    route_name: str
    max_start_altitude: int
    avalanche_start_zones: list
    kast: int
    dangers: str
    exposure: str
    difficulty: str
    comment: str
    equipment: str
    max_steepness: Any
    is_descent: bool
    start_location: str
    altitude: int
    geoJSON: dict

columns = {
  'Fjell': 'mountain_name',
  'Himmelretning (topp til bunn)': 'aspect',
  'Område': 'area',
  'Rute': 'route_name',
  'Høyeste startsted': 'max_start_altitude',
  'Løsneområder (obligatoriske)': 'avalanche_start_zones',
  'KAST': 'kast',
  'Terrengfeller/farer': 'dangers',
  'Eksponering': 'exposure',
  'Vanskelighetsgrad': 'difficulty',
  'Kommentar': 'comment',
  'Utstyr': 'equipment',
  'Max bratthet': 'max_steepness',
  'Primært nedkjøring?': 'is_descent',
  'Startsted': 'start_location',
  'Toppunkt': 'altitude',
  'Hvem': 'who'
}

routes = routes.rename(columns=columns)
routes = routes.drop(labels='who', axis='columns')
routes = routes.dropna(how='all', axis='columns')
routes = routes.fillna('')
routes['route_id'] = routes.index

In [152]:
routes_dict = routes.to_dict(orient='records')

for route in routes_dict:
      for key, val in route.items():
        if val == "FALSE" or val == "USANN":
          route[key] = False
      
        if val == "TRUE" or val == "SANN":
          route[key] = True

      if route['avalanche_start_zones'] == False:
        route['avalanche_start_zones'] = []

      if route['geoJSON']:
        route['geoJSON'] = json.loads(route['geoJSON'])

      if route['aspect']:
        route['aspect'] = route['aspect'].split(' ')

      if route['avalanche_start_zones']:
        elevation_spans = route['avalanche_start_zones'].replace(',', '.').split('.')
        elevation_spans = [tuple(span.strip().split(':')) for span in elevation_spans]
        route['avalanche_start_zones'] = elevation_spans
      


In [153]:
valid_routes = []
invalid_count = 0
for route in routes_dict:
  try:
    r = Route(**route)
    valid_routes.append(route)
  except ValidationError as e:
    error_text = (
      f'{route["mountain_name"]} - {route["route_name"]}: \n'
      f'{e} \n'
      f'-------------'
    )
    invalid_count += 1
    print(error_text)
  


Høgenibba - Normalveien: 
2 validation errors for Route
kast
  value is not a valid integer (type=type_error.integer)
max_steepness
  value is not a valid integer (type=type_error.integer) 
-------------
Kolåstind - Normalveien: 
2 validation errors for Route
kast
  value is not a valid integer (type=type_error.integer)
max_steepness
  value is not a valid integer (type=type_error.integer) 
-------------
Grøtdalstind - Sydflanken: 
2 validation errors for Route
kast
  value is not a valid integer (type=type_error.integer)
max_steepness
  value is not a valid integer (type=type_error.integer) 
-------------
Eidskyrkja - Fra Skinnviksætra: 
1 validation error for Route
max_steepness
  value is not a valid integer (type=type_error.integer) 
-------------
Nivane - Normalveien: 
2 validation errors for Route
max_steepness
  value is not a valid integer (type=type_error.integer)
altitude
  value is not a valid integer (type=type_error.integer) 
-------------
Romedalstind - Normalveien: 
1 va

In [154]:
invalid_count

32

In [155]:
len(valid_routes)

51

In [156]:
with open('routes.json', 'w') as file:
    file.write(json.dumps(valid_routes))
valid_routes

960684, 62.031271]]}},
  'route_id': 63},
 {'fjell_id': '',
  'mountain_name': 'Hellefjellet',
  'route_name': 'Østsida',
  'altitude': '1239',
  'area': 'Kolåshalvøya',
  'is_descent': False,
  'aspect': ['SØ', 'Ø', 'S'],
  'start_location': 'Heimsætra',
  'max_start_altitude': '278',
  'avalanche_start_zones': [('819-825', '30'),
   ('975-1080', '34-38'),
   ('1030-1233', '34-46')],
  'max_steepness': '46',
  'dangers': 'Bekkedal i Bjønnadalen. ',
  'kast': 3.0,
  'exposure': 'E3',
  'difficulty': 'GV',
  'equipment': 'Stegjern og isøks',
  'comment': '',
  'geoJSON': {'type': 'Feature',
   'properties': {},
   'geometry': {'type': 'LineString',
    'coordinates': [[6.375203, 62.196725],
     [6.373744, 62.196765],
     [6.372328, 62.197586],
     [6.371899, 62.198687],
     [6.371942, 62.199307],
     [6.371083, 62.199968],
     [6.369581, 62.200888],
     [6.369882, 62.201929],
     [6.370268, 62.20271],
     [6.37074, 62.20347],
     [6.37104, 62.204191],
     [6.370611, 62.205751